In [3]:
# Automated labeling pipeline
import sys
sys.path.append('../src')

from utils.change_detection import generate_change_labels
from utils.anomaly_detection import detect_spectral_anomalies
from utils.rule_based_labeling import rule_based_labels

# Generate labels using multiple methods
patches_dir = "../data/processed/patches"

print("🔧 Generating automated labels...")

# Method 1: Change detection
change_labels = generate_change_labels(patches_dir, "../data/annotations")
print(f"✅ Change detection: {sum(change_labels.values())} change patches")

# Method 2: Anomaly detection  
anomaly_labels = detect_spectral_anomalies(patches_dir)
print(f"✅ Anomaly detection: {sum(anomaly_labels.values())} anomalous patches")

# Method 3: Rule-based
rule_labels = rule_based_labels(patches_dir)
print(f"✅ Rule-based: {sum(rule_labels.values())} mining patches")

# Ensemble voting
final_labels = {}
for patch in change_labels.keys():
    votes = [change_labels.get(patch, 0), 
             anomaly_labels.get(patch, 0), 
             rule_labels.get(patch, 0)]
    final_labels[patch] = 1 if sum(votes) >= 2 else 0  # Majority voting

print(f"🎯 Final labels: {sum(final_labels.values())} mining patches out of {len(final_labels)} total")

# Save labels
import json
with open("../data/annotations/automated_labels.json", "w") as f:
    json.dump(final_labels, f, indent=2)

print("💾 Automated labels saved!")

🔧 Generating automated labels...
✅ Change detection: 0 change patches


NameError: name 'os' is not defined

In [4]:
import numpy as np
import os
import json
from pathlib import Path

# Setup paths
PATCHES_DIR = "../data/processed/patches"
os.makedirs("../data/annotations", exist_ok=True)

def simple_change_detection(patches_dir):
    """Simple change detection based on NDWI variance"""
    labels = {}
    
    patch_files = [f for f in os.listdir(patches_dir) if f.endswith('.npy')]
    print(f"Found {len(patch_files)} patches")
    
    for patch_file in patch_files:
        try:
            patch_data = np.load(os.path.join(patches_dir, patch_file))
            
            # Simple rule: High NDWI variance = potential mining activity
            ndwi_band = patch_data[4]  # NDWI is band 4
            ndwi_variance = np.var(ndwi_band)
            
            # Label as mining if high variance (suggesting changes)
            labels[patch_file] = 1 if ndwi_variance > 0.01 else 0
            
        except Exception as e:
            print(f"Error processing {patch_file}: {e}")
            labels[patch_file] = 0
    
    return labels

print("🔧 Generating simple automated labels...")
labels = simple_change_detection(PATCHES_DIR)

mining_count = sum(labels.values())
total_count = len(labels)
print(f"🎯 Labeling complete: {mining_count}/{total_count} patches labeled as mining ({mining_count/total_count*100:.1f}%)")

# Save labels
with open("../data/annotations/automated_labels.json", "w") as f:
    json.dump(labels, f, indent=2)

print("💾 Labels saved to ../data/annotations/automated_labels.json")

# Quick validation - show some examples
print("\n🔍 Sample labeled patches:")
for i, (patch_file, label) in enumerate(list(labels.items())[:5]):
    print(f"  {patch_file} → {'MINING' if label == 1 else 'NO MINING'}")

🔧 Generating simple automated labels...
Found 16 patches
🎯 Labeling complete: 0/16 patches labeled as mining (0.0%)
💾 Labels saved to ../data/annotations/automated_labels.json

🔍 Sample labeled patches:
  S2_Yamuna_2023_01_patch_0_512.npy → NO MINING
  S2_Yamuna_2023_01_patch_768_0.npy → NO MINING
  S2_Yamuna_2023_01_patch_0_0.npy → NO MINING
  S2_Yamuna_2023_01_patch_768_256.npy → NO MINING
  S2_Yamuna_2023_01_patch_256_512.npy → NO MINING
